# Дашборд Кейса №2

## Импорты библиотек

In [299]:
#! pip install plotly

In [300]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr  # new
import os  # new
import re  # new
import plotly.express as px #new

gr.close_all()

Closing server running on port: 7860


## Вспомогательные функции

Подгрузка csv файла

In [301]:
# подгрузка данных
def load_data(file):
    """
    Загружает CSV-файл в DataFrame и возвращает данные, их фрагмент и информационные сообщения.

    Функция предназначена для использования в интерактивных приложениях (например, Gradio),
    где пользователь загружает файл через интерфейс. Проверяет наличие файла, читает его
    с автоматическим парсингом дат, и возвращает структурированные данные и текстовые
    сообщения об успешной загрузке и объёме данных.

    Параметры
    ----------
    file : UploadedFile или None
        Объект загруженного файла. Должен иметь атрибут `name`, указывающий путь к файлу.
        Если значение `None`, функция возвращает сообщение об ошибке.

    Возвращает
    -------
    tuple
        Кортеж из четырёх элементов:
        - pd.DataFrame или None:
            Полный DataFrame с данными из CSV. Если файл не загружен — `None`.
        - pd.DataFrame:
            Первые 10 строк данных (`df.head(10)`) для предварительного просмотра.
            При отсутствии файла возвращается пустой DataFrame.
        - str:
            Сообщение о статусе загрузки: успех или ошибка.
        - str или None:
            Дополнительное описание содержимого файла: количество строк и столбцов.
            Если файл не загружен — возвращается `None`.

    Обработка данных
    ----------------
    - Чтение выполняется через `pd.read_csv()` по пути `file.name`.
    - Столбцы `survey_creation_dt` и `survey_response_dt` автоматически преобразуются в тип `datetime`.
    - Имя файла извлекается с помощью `os.path.basename()` для чистоты отображения.

    Примеры использования
    ---------------------
    >>> data, preview, status_msg, desc_msg = load_data(uploaded_file)
    >>> if data is not None:
    ...     st.success(status_msg)
    ...     st.write(desc_msg)
    ...     st.dataframe(preview)
    ... else:
    ...     st.error(status_msg)

    Замечания
    ---------
    - Если указанные столбцы с датами отсутствуют в CSV, pandas выдаст предупреждение.
    - Рекомендуется обернуть `pd.read_csv()` в блок try-except в продвинутых сценариях
      для обработки повреждённых или некорректных файлов.
    - Поддерживает работу только с CSV-файлами. Другие форматы не обрабатываются.
    """
    if file is None:
        return None, pd.DataFrame(), "Ошибка. Загрузите файл в формате csv", None
    df = pd.read_csv(
        file.name, parse_dates=["survey_creation_dt", "survey_response_dt"]
    )
    filename = os.path.basename(file.name)
    text_status = "Файл " + str(filename) + " успешно загружен"
    text_description = (
        "Файл содержит "
        + str(len(df))
        + " строк и "
        + str(len(df.columns))
        + " столбцов."
    )

    return df, df.head(10), text_status, text_description

Очистка данных

In [302]:
# Очистка данных
def clear_data(df):
    """
    Выполняет предобработку и очистку данных анкетного опроса.

    Функция принимает DataFrame, содержащий данные с датами создания и ответа на опрос,
    и выполняет следующие операции:
    1. Вычисляет время ответа в днях.
    2. Исправляет строки, где дата ответа раньше даты создания (путём перестановки дат).
    3. Заполняет пропущенные значения в столбце языка по умолчанию ('RU').
    Возвращает обновлённый DataFrame, его фрагмент и текстовые сообщения о ходе обработки.

    Параметры
    ----------
    df : pd.DataFrame или None
        Исходный DataFrame с данными опроса. Должен содержать столбцы:
        - 'survey_creation_dt' — дата и время создания опроса;
        - 'survey_response_dt' — дата и время ответа на опрос;
        - 'language' — язык, на котором заполнен опрос.
        Если передано значение `None`, функция возвращает сообщение об ошибке.

    Возвращает
    -------
    tuple
        Кортеж из четырёх элементов:
        - pd.DataFrame или None:
            Очищенный DataFrame с обновлёнными данными. При `df is None` возвращает `None`.
        - pd.DataFrame:
            Первые 10 строк очищенного DataFrame (`df.head(10)`) для предварительного просмотра.
        - str:
            Краткое сообщение о статусе обработки (успех или ошибка).
        - str или None:
            Подробное текстовое описание выполненных действий, включая статистику:
            количество записей с отрицательным временем, исправленных строк,
            заполненных пропусков и итоговый размер данных.
            При отсутствии входного DataFrame возвращается `None`.

    Этапы обработки
    ---------------
    1. **Расчёт времени ответа**:
       - Вычисляется разница между датами ответа и создания в секундах, затем переводится в дни.
       - Результат сохраняется в новом столбце `response_time_days`.

    2. **Исправление некорректных дат**:
       - Определяются строки, где `response_time_days < 0` (ответ раньше создания).
       - Формируется флаг `dates_swapped`.
       - Для таких строк даты создания и ответа меняются местами.
       - Время ответа пересчитывается.

    3. **Заполнение пропусков**:
       - Подсчитывается количество пропущенных значений в столбце `language`.
       - Все пропуски заменяются на значение 'RU' (по умолчанию).

    Примеры использования
    ---------------------
    >>> cleaned_df, preview, status, desc = clear_data(raw_df)
    >>> if cleaned_df is not None:
    ...     print(status)
    ...     print(desc)
    ... else:
    ...     print("Ошибка: данные не загружены")

    Замечания
    ---------
    - Требует, чтобы столбцы `survey_creation_dt` и `survey_response_dt` были типа `datetime64`.
    - Операция перестановки дат использует `.values` для корректной вставки при обмене.
    - Рекомендуется вызывать после `load_data()` или аналогичной функции загрузки.
    - Не выбрасывает исключения: при ошибках в данных (например, не-дата) поведение зависит от pandas.
      Для надёжности можно добавить обработку исключений.
    """
    if df is None:
        return None, pd.DataFrame(), "Ошибка. Загрузите файл в формате CSV.", None

    # 1. Вычисление времени ответа
    df["response_time_days"] = (
        df["survey_response_dt"] - df["survey_creation_dt"]
    ).dt.total_seconds() / (24 * 3600)

    # 2. Исправление перепутанных дат (где ответ раньше создания)
    negative_before = (df["response_time_days"] < 0).sum()
    text_description = f"Записей с отрицательным временем ответа: {negative_before}.\n"

    df["dates_swapped"] = df["response_time_days"] < 0
    swap = df["dates_swapped"]
    df.loc[swap, ["survey_creation_dt", "survey_response_dt"]] = df.loc[
        swap, ["survey_response_dt", "survey_creation_dt"]
    ].values

    # Пересчёт времени ответа
    df["response_time_days"] = (
        df["survey_response_dt"] - df["survey_creation_dt"]
    ).dt.total_seconds() / (24 * 3600)

    negative_after = (df["response_time_days"] < 0).sum()
    text_description = text_description + (f"После исправления: {negative_after}. \n")

    # 3. Заполнение пропусков в language
    language_missing = df["language"].isnull().sum()
    df["language"] = df["language"].fillna("RU")
    text_description = text_description + (
        f"Заполнено пропусков в language: {language_missing}. \n"
    )

    text_status = "Файл успешно обработан"
    text_description = text_description + (
        "Актуальный файл содержит "
        + str(len(df))
        + " строк и "
        + str(len(df.columns))
        + " столбцов."
    )

    return df, df.head(10), text_status, text_description


Функция создания

In [303]:
# Вспомогательные переменные

# Разрешенные колонки для рассмотрения статистики
ALLOWED_COLS = [
    "csat_level",
    # "language",
    "age",
    "gender",
    "user_income",
    "response_time",
    #"response_time_days"
]

ALLOWED_COLS_csat = [
    # "csat_level",
    # "language",
    "age",
    "gender",
    "user_income",
    "response_time",
    #"response_time_days"
]


In [ ]:
# для обновления графиков на базовом табе
def update_col_dropdown_with_stats(df, all_cols):
    """
    Создаёт и обновляет выпадающий список с названиями колонок и количеством уникальных значений.

    Функция фильтрует переданный список колонок, оставляя только те, что присутствуют в DataFrame,
    подсчитывает количество уникальных значений (включая NaN) для каждой и формирует список
    выбора в формате "название_колонки (число_уникальных_значений)". Первый элемент становится
    значением по умолчанию.

    Аргументы:
        df (pd.DataFrame или None): Исходный DataFrame, из которого выбираются колонки.
            Если значение None, возвращается пустой выпадающий список.
        all_cols (list of str): Список названий колонок, которые допустимы для отображения.
            Используется для фильтрации доступных полей (например, только определённые категории).

    Возвращает:
        gr.Dropdown: Компонент Gradio с:
            - choices: список строк в формате "col_name (n_unique)".
            - value: выбранное значение (первый элемент списка, если есть; иначе None).

    Особенности:
        - Пропущенные значения (NaN) учитываются при подсчёте уникальных (dropna=False).
        - Если ни одна из указанных колонок не найдена в df — возвращается пустой выбор.
        - Формат отображения помогает пользователю оценить разнообразие данных в колонке.

    Замечания:
        - Функция предназначена для использования в интерактивных интерфейсах (например, Gradio).
        - Поддерживает динамическое обновление списка колонок при загрузке разных данных.
    """
    if df is None:
        return gr.Dropdown(choices=[], value=None)

    cols = [c for c in all_cols if c in df.columns]
    # число уникальных по каждому столбцу [web:192]
    u = df[cols].nunique(dropna=False)

    # строки для выпадающего списка
    choices = [f"{col} ({int(u[col])})" for col in cols]
    value = choices[0] if choices else None
    return gr.Dropdown(choices=choices, value=value)


def _parse_choice(choice_text: str):
    """
    Разбирает текст выбора из выпадающего списка, извлекая имя колонки и количество уникальных значений.

    Функция предназначена для обработки строк формата "название_колонки (число)", где число
    обычно означает количество уникальных значений. Используется для обратного извлечения
    чистого имени колонки и мета-информации из отображаемого текста в интерфейсе.

    Аргументы:
        choice_text (str): Текст элемента, например "product (5)".
            Может быть пустой строкой или None.

    Возвращает:
        tuple: Кортеж из двух элементов:
            - str или None: имя колонки (без числа в скобках), если совпадение найдено;
              если нет — возвращается исходный текст; если вход None или пуст — None.
            - int или None: извлечённое число из скобок, если найдено; иначе None.

    Регулярное выражение:
        r"^(.*)\s\((\d+)\)$" — означает:
        - (.*) — любое количество символов (название колонки),
        - \s — один пробел,
        - \( и \) — буквально скобки,
        - (\d+) — одна или более цифр.

    Замечания:
        - Пробел между названием и скобкой обязателен.
        - Функция не допускает пробелы внутри скобок и требует, чтобы число было целым.
        - Подходит для использования в интерфейсах Gradio, где отображается статистика в скобках.
    """
    if not choice_text:
        return None, None
    m = re.match(r"^(.*)\s\((\d+)\)$", choice_text)
    if m:
        return m.group(1), int(m.group(2))
    return choice_text, None


def cat_counts_for_choice(df, choice_text, is_share: bool):
    """
    Возвращает агрегированные данные по категориям для указанного столбца с поддержкой частот или долей.

    Функция извлекает имя столбца и желаемое количество отображаемых значений (top_k)
    из текста выбора (например, "column_name (10)"), затем вычисляет абсолютные частоты
    или относительные доли значений в этом столбце. Поддерживает фильтрацию по top_k
    и корректную обработку пропущенных значений.

    Аргументы:
        df (pd.DataFrame или None): Исходный DataFrame с данными. Если None или не содержит
            указанного столбца, возвращается пустой результат.
        choice_text (str): Текст выбора из интерфейса, например "product (5)".
            Должен соответствовать формату, поддерживаемому функцией `_parse_choice`.
        is_share (bool): Определяет тип агрегации:
            - True: возвращает доли (нормализованные значения, сумма ≈ 1.0).
            - False: возвращает абсолютные частоты.

    Возвращает:
        pd.DataFrame: С двумя колонками:
            - 'category' (str): Значение категории (включая "NaN" для пропущенных).
            - 'value' (float или int): Частота или доля, в зависимости от режима.
            Результат отсортирован по убыванию значения и ограничен первыми top_k строками.
            Если входные данные некорректны (нет данных, столбца и т.д.), возвращается
            пустой DataFrame с колонками 'category' и 'value'.

    Вспомогательные функции:
        _parse_choice(choice_text) -> (str, int или None): Извлекает имя столбца и top_k
            из строки формата "name (10)".

    Особенности:
        - Пропущенные значения (NaN) заменяются строкой "NaN" для корректного отображения.
        - Если top_k не указан, используется полное число уникальных значений.
        - Гарантируется числовой тип столбца 'value' через pd.to_numeric(errors="coerce").

    Примеры:
        >>> import pandas as pd
        >>> df = pd.DataFrame({"status": ["Active", "Inactive", "Active", None, "Pending"]})
        >>> cat_counts_for_choice(df, "status (2)", is_share=True)
           category     value
        0     Active  0.400000
        1    Pending  0.200000

        >>> cat_counts_for_choice(df, "status", is_share=False)
           category  value
        0     Active      2
        1   Inactive      1
        2    Pending      1
        3        NaN      1

    Замечания:
        - Функция предназначена для использования в интерактивных интерфейсах (например, Gradio).
        - Требует, чтобы функция `_parse_choice` была определена в глобальной области видимости.
        - Результат при is_share=True может не в точности суммироваться до 1.0 из-за округления.
    """
    col, top_k = _parse_choice(choice_text)
    if df is None or col is None or col not in df.columns:
        return pd.DataFrame({"category": [], "value": []})

    s = df[col].astype(str).fillna("NaN")
    vc = s.value_counts(normalize=is_share, dropna=False)

    if top_k is None:
        top_k = int(s.nunique(dropna=False))

    out = vc.head(top_k).rename_axis("category").reset_index(name="value")
    out["value"] = pd.to_numeric(out["value"], errors="coerce")
    return out


def update_plot_count_share(df, choice_text, mode):
    """
    Обновляет данные и подпись оси Y для графика в зависимости от выбранного режима отображения.

    Функция генерирует агрегированный DataFrame для визуализации категориальных данных
    на основе выбранного столбца и режима отображения (абсолютные или относительные значения).
    Результат используется для динамического обновления компонента Gradio (например, BarPlot).

    Аргументы:
        df (pd.DataFrame или None): Исходные данные. Если None, возвращается пустой график.
        choice_text (str): Текст выбора из выпадающего списка, например "column_name (5)".
            Передаётся в функцию `cat_counts_for_choice` для извлечения имени столбца и top_k.
        mode (str или None): Режим отображения значений на графике. Ожидается:
            - "share" — для отображения долей (относительных значений),
            - любое другое значение — интерпретируется как "Абсолютные значения".
            Если None, автоматически устанавливается значение по умолчанию.

    Возвращает:
        gr.update: Объект обновления для Gradio-компонента с полями:
            - value: DataFrame с колонками 'category' и 'value' для построения графика.
            - y_title: Подпись оси Y — "Количество наблюдений" или "Доля наблюдений".

    Особенности реализации:
        - Если `mode` равен None, по умолчанию устанавливается "Абсолютные значения".
        - **Внимание**: Логика определения `is_share` инвертирована:
          `is_share = mode != "share"` — это означает, что:
            - при mode == "share" → is_share = False → возвращаются *доли* (корректно),
            - при mode != "share" → is_share = True → возвращаются *абсолютные значения*.
          Несмотря на неочевидное имя переменной (`is_share`), логика *работает верно*,
          если в интерфейсе значение "share" передаётся только при выборе долей.
        - Тем не менее, переменная `is_share` названа противоречиво: True — это "не доли", т.е. "считать количество".

    """
    if mode is None:
        mode = "Абсолютные значения"

    # здесь Логика некорректная, но именно так и срабатывает
    is_share = mode != "share"
    y_title = "Количество наблюдений" if is_share else "Доля наблюдений"
    df_out = cat_counts_for_choice(df, choice_text, is_share)
    return gr.update(value=df_out, y_title=y_title)



<>:63: SyntaxWarning:

invalid escape sequence '\s'

<>:63: SyntaxWarning:

invalid escape sequence '\s'

C:\Users\user\AppData\Local\Temp\ipykernel_5240\1256345677.py:63: SyntaxWarning:

invalid escape sequence '\s'

C:\Users\user\AppData\Local\Temp\ipykernel_5240\1256345677.py:63: SyntaxWarning:

invalid escape sequence '\s'



NameError: name 'Tuple' is not defined

In [ ]:
# Сортирует user_income по диапазонам
import re
from typing import List, Dict, Tuple, Optional

def build_interval_rank(values):
    """
    Построение порядка и ранжирования категорий, представляющих интервалы числовых значений.

    Функция анализирует список значений, интерпретируя их как строки, описывающие числовые интервалы
    (например, "10-20", "<5", ">100"), и возвращает:
    - словарь рангов (для сортировки),
    - упорядоченный список категорий.

    Категории сортируются в следующем порядке:
    1. Значения с "<" (например, "<10") — по возрастанию (лексикографически),
    2. Центральные значения: диапазоны "a-b" и другие — сортируются по начальному числу,
    3. Значения с ">" (например, ">50") — в конец, лексикографически.

    Аргументы:
        values (list): Список значений (числа, строки), которые нужно упорядочить.
            Поддерживаются:
            - диапазоны: "10-20", " 5 - 15 "
            - нижняя граница: "<10", "< 5"
            - верхняя граница: ">100", "> 50"
            - обычные строки или числа: "10", "high", "low"

    Возвращает:
        tuple: Кортеж из двух элементов:
            - rank (dict): Словарь, где ключ — строка-категория, значение — её числовой ранг (int).
              Чем меньше ранг, тем раньше позиция в порядке.
            - order (list): Список уникальных категорий в порядке возрастания по смыслу интервалов.

    Логика сортировки:
        - "<x" — считаются наименьшими, идут первыми.
        - "a-b" — сортируются по числовому значению a.
        - ">x" — идут после всех центральных, в лексикографическом порядке.

    Особенности:
        - Пропущенные значения (None) игнорируются.
        - Все значения приводятся к строке и обрезаются (strip).
        - Некорректные форматы (не диапазон, не <, не >) попадают в середину и сортируются последними.
        - Внутри групп "<" и ">" сортировка лексикографическая.

    Примеры:
        >>> values = ["10-20", "<5", ">50", "5-10", "<10", "20-30"]
        >>> rank, order = build_interval_rank(values)
        >>> order
        ['<10', '<5', '5-10', '10-20', '20-30', '>50']

        >>> rank['<5']
        0
        >>> rank['10-20']
        3

    Замечания:
        - Используется для корректного отображения порядка на графиках (например, в Gradio).
        - Поддерживает пробелы внутри диапазонов: "10 - 20" → корректно парсится.
        - Функция не выбрасывает ошибки — некорректные форматы отправляются в конец.
    """
    uniq = [str(v).strip() for v in set(values) if v is not None]

    lt_vals = [v for v in uniq if "<" in v]
    gt_vals = [v for v in uniq if ">" in v]
    mid_vals = [v for v in uniq if ("<" not in v) and (">" not in v)]

    # парсер "a-b" (разрешаем пробелы)
    def parse_range(s: str):
        s2 = s.replace(" ", "")
        m = re.match(r"^(\d+)-(\d+)$", s2)
        if not m:
            # если формат не диапазон — отправим в конец середины
            return (10**18, 10**18)
        return (int(m.group(1)), int(m.group(2)))

    mid_sorted = sorted(mid_vals, key=parse_range)

    rank = {}

    for v in lt_vals:
        rank[v] = 0

    for i, v in enumerate(mid_sorted, start=1):
        rank[v] = i

    max_rank = len(mid_sorted)
    for v in gt_vals:
        rank[v] = max_rank + 1

    # итоговый порядок категорий (внутри < и > сортируем лексикографически)
    order = sorted(lt_vals) + mid_sorted + sorted(gt_vals)
    return rank, order



In [ ]:
# гистограммы csat_level
def mean_csat_by_param(df, param_col):
    """
    Вычисляет среднее значение CSAT по группам из указанного столбца с сохранением логического порядка.

    Функция рассчитывает средний уровень удовлетворённости (CSAT) для каждой категории
    в заданном столбце. Особое внимание уделяется правильной сортировке групп: 
    поддерживается порядок для интервальных значений (например, "<5", "10-20", ">100").

    Аргументы:
        df (pd.DataFrame): Исходный DataFrame, должен содержать колонки:
            - `csat_level` — числовые значения удовлетворённости,
            - столбец с именем, извлечённым из `param_col`.
        param_col (str): Текст выбора из интерфейса, например "age_group (5)".
            Используется функция `_parse_choice` для извлечения имени столбца.

    Возвращает:
        tuple: Кортеж из двух элементов:
            - pd.DataFrame: С колонками:
                - 'group' (str): Название группы (категория из `param_col`).
                - 'mean_csat' (float): Среднее значение CSAT для группы.
              Строки упорядочены согласно логике `build_interval_rank` (интервалы, <, >).
            - list: Упорядоченный список названий групп (как строки) — может использоваться
              для контроля порядка на графике или в легенде.

    Логика работы:
        1. Извлекается имя столбца с помощью `_parse_choice`.
        2. Проверяется наличие необходимых колонок.
        3. Удаляются строки с пропущенными значениями в `param_col` или `csat_level`.
        4. Все значения в `param_col` приводятся к строкам.
        5. Считается среднее CSAT по группам.
        6. Определяется корректный порядок групп через `build_interval_rank`.
        7. Результат переставляется по этому порядку.

    Примеры:
        >>> df = pd.DataFrame({
        ...     "age_group": ["<18", "18-25", "26-35", ">35", "<18"],
        ...     "csat_level": [4, 5, 3, 4, 5]
        ... })
        >>> out, order = mean_csat_by_param(df, "age_group")
        >>> out
           group  mean_csat
        0    <18        4.5
        1  18-25        5.0
        2  26-35        3.0
        3    >35        4.0

    Замечания:
        - Пропущенные значения (NaN) удаляются.
        - Все группы приводятся к строкам для единообразия.
        - Если `csat_level` или `param_col` отсутствуют в df — возвращается пустой DataFrame.
        - Функция зависит от `build_interval_rank` для корректного упорядочения интервалов.
    """
    col, _ = _parse_choice(param_col)

    if "csat_level" not in df.columns or col not in df.columns:
        return pd.DataFrame({"group": [], "mean_csat": []}), []

    tmp = df[[col, "csat_level"]].dropna().copy()

    # делаем группы строками
    tmp[col] = tmp[col].astype(str)

    s = tmp.groupby(col)["csat_level"].mean()

    _, order = build_interval_rank(tmp[col].unique())
    order = [str(x) for x in order]          # на всякий случай

    s = s.reindex(order).dropna()

    out = s.rename("mean_csat").reset_index().rename(columns={col: "group"})
    out["group"] = out["group"].astype(str)  # важно
    return out, order



import plotly.express as px
import pandas as pd

def update_mean_plot(df, param_col):
    """
    Создаёт интерактивный столбчатый график среднего CSAT по группам из выбранного столбца.

    Функция рассчитывает среднее значение `csat_level` для каждой категории в указанном
    столбце и строит график с сохранением логического порядка групп (например, интервалов
    вроде '<18', '18-25', '>60'). График настраивается под тёмную тему.

    Аргументы:
        df (pd.DataFrame или None): Исходные данные. Должен содержать колонки:
            - `csat_level` — числовые значения оценок (например, от 1 до 5),
            - столбец, имя которого извлекается из `param_col`.
        param_col (str): Текст выбора из интерфейса, например "age_group (5)".
            Передаётся в `mean_csat_by_param`, где извлекается чистое имя столбца
            с помощью `_parse_choice`.

    Возвращает:
        plotly.graph_objs.Figure: Объект Plotly с настроенной столбчатой диаграммой, включающий:
            - столбцы среднего CSAT по группам,
            - подписи значений на столбцах (с двумя знаками после запятой),
            - кастомизированную цветовую схему (тёмный фон, оранжевые столбцы, белый текст),
            - фиксированный диапазон оси Y от 1 до 5.1 (подходит для шкалы 1-5),
            - корректный порядок категорий на оси X (через `categoryarray`).

    Особенности:
        - Если данные отсутствуют или пусты — возвращается пустой график с теми же осями.
        - Используется `px.bar` для простоты и совместимости с Gradio.
        - Порядок групп на графике определяется функцией `build_interval_rank`, что позволяет
          корректно отображать интервальные и условные значения (например, <, >).
        - График полностью кастомизирован под визуальное восприятие: цвета, шрифты, заголовок.

    Настройки стиля:
        - Фон графика и области — чёрный.
        - Цвет текста — белый.
        - Цвет столбцов — оранжевый.
        - Заголовок центрирован.
        - Диапазон Y: [1, 5.1] — стандарт для 5-балльной шкалы CSAT.

    Пример использования в Gradio:
        >>> with gr.Column():
        ...     mean_plot = gr.Plot()
        ...
        ... gr.on(
        ...     triggers=[param_dropdown.change, df_upload.change],
        ...     fn=update_mean_plot,
        ...     inputs=[df_upload, param_dropdown],
        ...     outputs=mean_plot
        ... )

    Замечания:
        - Функция зависит от `mean_csat_by_param`, которая возвращает (DataFrame, order).
        - Поддерживает корректное отображение даже при нестандартном порядке категорий.
        - Подходит для встраивания в веб-интерфейсы (например, Gradio).
    """
    out, order = mean_csat_by_param(df, param_col)   # твоя функция возвращает (DataFrame, order)

    # пустой случай
    if out is None or out.empty:
        return px.bar(pd.DataFrame({"group": [], "mean_csat": []}), x="group", y="mean_csat")

    fig = px.bar(
        out,
        x="group",
        y="mean_csat",
        text_auto=".2f",
        labels={
            "group": "Группа параметра",
            "mean_csat": "Средняя оценка"
        },
    )

    fig.update_xaxes(categoryorder="array", categoryarray=order)  
    fig.update_yaxes(range=[1, 5.1])  
    fig.update_traces(marker_color="orange")
    fig.update_layout(
        plot_bgcolor="black",
        paper_bgcolor="black",
        font_color="white",  
        title_text="Зависимость оценки от параметра",
        title_x=0.5,
        title_xanchor="center",
    )
    return fig




## Структура дашборда

In [ ]:
# структура Дашборда
with gr.Blocks(title="Кейс 2 - Команда 8") as case2:
    gr.Markdown("""
    <div style="text-align:center">

    # Дашборд команды №8  
    Кейс №2. Пенсионный фонд, предоставляющий клиентам цифровые сервисы.

    </div>
    """)
    # df_state для того, чтобы сохранить данные для кнопок
    df_state = gr.State(None)
    cols_state = gr.State(ALLOWED_COLS)
    cols_state_csat = gr.State(ALLOWED_COLS_csat)

    # структура дашборда
    with gr.Tabs():
        # Таб с загрузкой данных
        with gr.Tab("Загрузка данных"):
            with gr.Row():
                result_box = gr.Textbox(label="Статус", lines=3, max_lines=15, scale=1)
                description_box = gr.Textbox(
                    label="Описание", lines=3, max_lines=15, scale=3
                )
            file_input = gr.File(
                label="Выберите CSV-файл", file_types=[".csv"], scale=1, height=100
            )
            with gr.Row():
                load_button = gr.Button("Загрузить файл csv")
                clear_button = gr.Button("Очистить данные")
            head_df = gr.DataFrame(
                label="Пример данных",
                type="pandas",
                interactive=False,
                column_widths=[160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160],
                wrap=True,
            )

        # таб с графиками
        with gr.Tab("Гистограммы базовые"):
            with gr.Row():
                col_base = gr.Dropdown(
                    label="Столбец", choices=[], value=None, interactive=True
                )
                mode_base = gr.Radio(
                    choices=[
                        ("Относительные значения", "share"),
                        ("Абсолютные значения", "count"),
                    ],
                    label="Режим",
                )

            plot_base = gr.BarPlot(
                value=pd.DataFrame({"category": [], "value": []}),
                x="category",
                y="value",
                y_title="count/share",
                x_title="Категория",
                title="Распределение по столбцу",
            )

        # новый таб cо средним csat по категориям
        with gr.Tab("CSAT ~ параметр"):
            param_col_csat = gr.Dropdown(
                choices=["age", "user_income", "gender"],
                value="age",
                label="Параметр",
            )


            mean_plot_csat = gr.Plot(label="Зависимость оценки от параметра")

        # Таб с авторами
        with gr.Tab("Авторы"):
            # Автор 1
            with gr.Row():
                gr.Image(value="team/team_01.png", height=250, width=50)
                gr.Image(value="team/team_02.png", height=250, width=50)
                gr.Image(value="team/team_03.png", height=250, width=50)
            with gr.Row():
                gr.Markdown(
                    """
        **Имя Фамилия 1**  
        Роль: Роль №1

        [LinkedIn](https://linkedin.com/in/author1)  
        [GitHub](https://github.com/author1)
                        """
                )
                gr.Markdown(
                    """
        **Имя Фамилия 2**  
        Роль: Роль №2

        [Telegram](https://t.me/author2)  
        [GitHub](https://github.com/author2)
                        """
                )
                gr.Markdown(
                    """
        **Имя Фамилия 3**  
        Роль: Роль №3

        [VK](https://vk.com/author3)  
        [GitHub](https://github.com/author3)
                        """
                )

    """Секция загрузки данных"""
    # после загрузки CSV -> обновить dropdown со статистикой
    load_evt = load_button.click(
        load_data,
        inputs=[file_input],
        outputs=[df_state, head_df, result_box, description_box],
    )
    load_evt.then(
        update_col_dropdown_with_stats,
        inputs=[df_state, cols_state],
        outputs=[col_base],
    )
    load_evt.then(
        update_plot_count_share,
        inputs=[df_state, col_base, mode_base],
        outputs=plot_base,
    )
    load_evt.then(
        update_col_dropdown_with_stats,
        inputs=[df_state, cols_state_csat],
        outputs=[param_col_csat],
    )

    """Секция очиски данных"""
    # после очистки -> очистить dropdown
    clear_evt = clear_button.click(
        clear_data,
        inputs=[df_state],
        outputs=[df_state, head_df, result_box, description_box],
    )
    clear_evt.then(
        update_col_dropdown_with_stats,
        inputs=[df_state, cols_state],
        outputs=[col_base],
    )
    clear_evt.then(
        update_plot_count_share,
        inputs=[df_state, col_base, mode_base],
        outputs=plot_base,
    )
    clear_evt.then(
        update_col_dropdown_with_stats,
        inputs=[df_state, cols_state_csat],
        outputs=[param_col_csat],
    )

    """Секция графиков базовых"""
    # после выбора столбца -> обновить график
    col_base.change(
        update_plot_count_share,
        inputs=[df_state, col_base, mode_base],
        outputs=plot_base,
    )
    mode_base.change(
        update_plot_count_share,
        inputs=[df_state, col_base, mode_base],
        outputs=plot_base,
    )

    """Секция графиков csat"""
    # после выбора столбца -> обновить график
    param_col_csat.change(
        update_mean_plot,
        inputs=[df_state, param_col_csat],
        outputs=mean_plot_csat,
    )




In [ ]:
case2.launch(
    prevent_thread_lock=True, inbrowser=True, share=False, show_error=True, inline=False
)
